In [ ]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset


dataset_name = "Babelscape/multinerd"

# Load the dataset
dataset = load_dataset(dataset_name)

# Accessing the train split
train_dataset = dataset['train']

# Accessing data samples
for example in train_dataset[:5]:
    print(example)


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokens
ner_tags
lang


In [ ]:
# Print information about the dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 2678400
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 334800
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 335986
    })
})


In [ ]:
# Filtering out non-English examples from the training set
english_train_set = dataset['train'].filter(lambda example: example['lang'] == 'en')


Filter:   0%|          | 0/2678400 [00:00<?, ? examples/s]

In [ ]:
english_train_set[0]["tokens"]

['The', 'type', 'locality', 'is', 'Kīlauea', '.']

In [ ]:
english_train_set[0]["ner_tags"]

[0, 0, 0, 0, 5, 0]

In [ ]:
label_names = {

    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-DIS",
    8: "I-DIS",
    9: "B-ANIM",
    10: "I-ANIM",
    11: "B-TIME",
    12: "I-TIME",
    13: "B-VEHI",
    14: "I-VEHI",
    15: "B-MYTH",
    16: "I-MYTH",
    17: "B-PLANT",
    18: "I-PLANT",
    19: "B-MEDIA",
    20: "I-MEDIA",
    21: "B-EVE",
    22: "I-EVE",
    23: "B-FOOD",
    24: "I-FOOD",
    25: "B-INST",
    26: "I-INST",
    27: "B-BIO",
    28: "I-BIO",
    29: "B-CEL",
    30: "I-CEL"
}

words = english_train_set[0]["tokens"]
labels = english_train_set[0]["ner_tags"]
line1 = ""
line2 = ""

for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


The type locality is Kīlauea . 
O   O    O        O  B-LOC   O 


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(english_train_set[0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'The',
 'type',
 'locality',
 'is',
 'K',
 '##ī',
 '##lau',
 '##ea',
 '.',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of neww word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token..
            new_labels.append(-100)
        else:
            # word(same) as a previous token..
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = english_train_set[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 5, 0]
[-100, 0, 0, 0, 0, 5, 6, 6, 6, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = english_train_set.map(
    tokenize_and_align_labels,
    batched=True,
)



Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

**FINE TUNING WITH KERAS:**

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Example of labels...
labels_list = [
    [-100, 0, 0, 0, 0, 5, 6, 6, 6, 0, -100],
    [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 26, 26, 26, 26, 0, 0, 0, -100]
]

# Pad sequences....
padded_labels = pad_sequence([torch.tensor(seq) for seq in labels_list], batch_first=True, padding_value=-100)

print(padded_labels)


tensor([[-100,    0,    0,    0,    0,    5,    6,    6,    6,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,   25,   26,   26,   26,   26,   26,   26,    0,
            0,    0,    0,    0,    0,    0,    0,   25,   26,   26,   26,   26,
           26,   26,   26,    0,    0,    0, -100]])


In [ ]:
tf_train_dataset = tokenized_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


**Defining the Model**

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

31

**Fine-tuning the model**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import create_optimizer
import tensorflow as tf


tf.keras.mixed_precision.set_global_policy("mixed_float16")

num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
pip install tensorflow --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.14.0
    Uninstalling tensorflow-2.14.0:
      Successfully un

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


**Metrics**

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 636.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=38440d89123a4fe85f049a3f42530f9c3f9fd802a1779bded2a49b2077ec8475
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 960.9 kB/s eta 0:00:00


In [ ]:
import evaluate


In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
tok = english_train_set[3]["tokens"]
print(tok)

labels = english_train_set[3]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['Created', 'by', 'James', 'Cameron', 'and', 'Charles', 'H.', 'Eglee', ',', 'it', 'stars', 'Jessica', 'Alba', 'in', 'her', 'breakthrough', 'role', '.']


['O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'B-PER',
 'I-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'PER': {'precision': 0.6666666666666666,
  'recall': 0.6666666666666666,
  'f1': 0.6666666666666666,
  'number': 3},
 'overall_precision': 0.6666666666666666,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.6666666666666666,
 'overall_accuracy': 0.9444444444444444}

In [ ]:
from transformers import pipeline
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Hina Faizan and I am doing my research on NLP.")

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'entity_group': 'PER',
  'score': 0.99922717,
  'word': 'Hina Faizan',
  'start': 11,
  'end': 22},
 {'entity_group': 'MISC',
  'score': 0.8928512,
  'word': 'NLP',
  'start': 53,
  'end': 56}]

PART 2 PERSON(PER), ORGANIZATION(ORG), LOCATION(LOC), DISEASES(DIS),
ANIMAL(ANIM)bold text

In [ ]:
pip install datasets


In [ ]:
from datasets import load_dataset

# Specify the dataset name
dataset_name = "Babelscape/multinerd"

# Load the dataset
dataset = load_dataset(dataset_name)

# Access the train split (you can change it to 'test', 'validation', etc.)
train_dataset = dataset['train']

# Access data samples
for example in train_dataset[:5]:
    print(example)


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

tokens
ner_tags
lang


In [ ]:
# Filter out non-English examples from the training set
english_train_set = dataset['train'].filter(lambda example: example['lang'] == 'en')


In [ ]:
english_train_set[0]["tokens"]

['The', 'type', 'locality', 'is', 'Kīlauea', '.']

In [ ]:
english_train_set[0]["ner_tags"]

[0, 0, 0, 0, 5, 0]

In [ ]:
label_names = {

    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-DIS",
    8: "I-DIS",
    9: "B-ANIM",
    10: "I-ANIM",

}

words = english_train_set[0]["tokens"]
labels = english_train_set[0]["ner_tags"]
line1 = ""
line2 = ""

for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)


The type locality is Kīlauea . 
O   O    O        O  B-LOC   O 


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
inputs = tokenizer(english_train_set[0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'The',
 'type',
 'locality',
 'is',
 'K',
 '##ī',
 '##lau',
 '##ea',
 '.',
 '[SEP]']

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:

            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:

            new_labels.append(-100)
        else:

            label = labels[word_id]

            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = english_train_set[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 5, 0]
[-100, 0, 0, 0, 0, 5, 6, 6, 6, 0, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = english_train_set.map(
    tokenize_and_align_labels,
    batched=True,
)



In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Example of labels....
labels_list = [
    [-100, 0, 0, 0, 0, 5, 6, 6, 6, 0, -100],
    [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 26, 26, 26, 26, 0, 0, 0, -100]
]

# Pad sequences....
padded_labels = pad_sequence([torch.tensor(seq) for seq in labels_list], batch_first=True, padding_value=-100)

print(padded_labels)


tensor([[-100,    0,    0,    0,    0,    5,    6,    6,    6,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,   25,   26,   26,   26,   26,   26,   26,    0,
            0,    0,    0,    0,    0,    0,    0,   25,   26,   26,   26,   26,
           26,   26,   26,    0,    0,    0, -100]])


In [ ]:
tf_train_dataset = tokenized_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

11

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import create_optimizer
import tensorflow as tf


tf.keras.mixed_precision.set_global_policy("mixed_float16")


num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [ ]:
# Accessing the individual splits.....
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
pip install tensorflow --upgrade


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [ ]:
import evaluate


In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
english_train_set[4]["tokens"]

['The',
 'song',
 'was',
 'written',
 'by',
 'Sarah',
 'Buxton',
 ',',
 'Rodney',
 'Clawson',
 'and',
 'Chris',
 'Tompkins',
 '.']

In [ ]:
labels = english_train_set[4]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'B-PER',
 'I-PER',
 'O']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
from transformers import pipeline


model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Hina and I working as NLP researcher. ")

[{'entity_group': 'PER',
  'score': 0.9984766,
  'word': 'Hina',
  'start': 11,
  'end': 15},
 {'entity_group': 'ORG',
  'score': 0.9800407,
  'word': 'NLP',
  'start': 33,
  'end': 36}]